In [3]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder

import os
import pickle
from datetime import datetime
from dateutil.parser import parse

%matplotlib inline


In [5]:

with open("App/randForest_best_model.pkl", 'rb') as picklefile:
    PREDICTOR = pickle.load(picklefile)



In [6]:
df_ad = pd.read_csv('data/test.csv')
df_ad

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00
5,5,110769,18,1,13,107,2017-11-10 04:00:00
6,6,12540,3,1,1,137,2017-11-10 04:00:00
7,7,88637,27,1,19,153,2017-11-10 04:00:00
8,9,14932,18,1,10,107,2017-11-10 04:00:00
9,8,123701,12,1,53,424,2017-11-10 04:00:00


In [7]:
def convertDF(df_old):
    df_old['click_time'] = pd.to_datetime(df_old['click_time'], format='%Y/%m/%d %H:%M:%S')
    df_old['hour']=df_old['click_time'].dt.hour
    df_old['date']=df_old['click_time'].dt.date
    df_old['day']=df_old['click_time'].dt.day
    df_old['month']=df_old['click_time'].dt.month
    df_old['year']=df_old['click_time'].dt.year
    return df_old

In [8]:
df_new = convertDF(df_ad)
df_new.head()

,click_id,ip,app,device,os,channel,click_time,hour,date,day,month,year
0,0,5744,9,1,3,107,2017-11-10 04:00:00,4,2017-11-10,10,11,2017
1,1,119901,9,1,3,466,2017-11-10 04:00:00,4,2017-11-10,10,11,2017
2,2,72287,21,1,19,128,2017-11-10 04:00:00,4,2017-11-10,10,11,2017
3,3,78477,15,1,13,111,2017-11-10 04:00:00,4,2017-11-10,10,11,2017
4,4,123080,12,1,13,328,2017-11-10 04:00:00,4,2017-11-10,10,11,2017


In [9]:
df_new.columns

Index(['click_id', 'ip', 'app', 'device', 'os', 'channel', 'click_time',
       'hour', 'date', 'day', 'month', 'year'],
      dtype='object')

In [14]:
x = df_new.loc[:, ['ip', 'app', 'device', 'os', 'channel', 'hour']]

In [ ]:
score = PREDICTOR.predict_proba(x)
score

In [57]:
score[:, 0]

array([0.72190481, 0.84932196, 0.99214286, ..., 0.95484957, 0.94093056,
       0.9909902 ])

In [72]:
df_new_sub = df_new

In [73]:
df_new_sub['is_attributed'] = score[:, 1] > 0.5
df_new_sub['is_attributed'] = df_new_sub['is_attributed'].astype(int)

In [74]:
df_new_sub = df_new_sub.drop(['ip', 'app', 'device', 'os', 'channel', 'click_time',
       'hour', 'date', 'day', 'month', 'year'], axis=1)

In [75]:
# df_new_sub.rename(columns={"click_id": "item_id"})

In [77]:
len(df_new_sub['is_attributed'] == 1)

18790469

In [78]:
df_new_sub.to_csv('submission.csv', index=False)

NameError: name 'PREDICTORCat' is not defined